# (3) Model Selection and Predictions
In this notebook, we will choose our model, generate our predictions and make some recommendations to improve the sale price.

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# imports
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error

In [3]:
# load datasets
train = pd.read_csv('datasets/train_sel.csv')
test = pd.read_csv('datasets/test_sel.csv')

# check data loaded correctly
train.head()

,Total_Bsmt_SF,Gr_Liv_Area,Garage_Area,Year_Built,Year_Remod/Add,Mas_Vnr_Area,Full_Bath,BsmtFin_SF_1,Overall_Qual,Exter_Qual,...,Fireplace_Qu,Garage_Finish,Bsmt_Exposure,Heating_QC,MS_SubClass,Foundation,Garage_Type,Neighborhood_High,Neighborhood_Low,SalePrice
0,725.0,1479,475.0,1976,2005,289.0,2,533.0,6.0,1,...,0,2,0,2,1,0,1,0,0,130500
1,913.0,2122,559.0,1996,1997,132.0,2,637.0,7.0,1,...,2,2,0,2,1,1,1,0,0,220000
2,1057.0,1057,246.0,1953,2007,0.0,1,731.0,5.0,0,...,0,1,0,0,1,0,0,0,0,109000
3,384.0,1444,400.0,2006,2007,0.0,2,0.0,5.0,0,...,0,3,0,1,1,1,1,0,0,174000
4,676.0,1445,484.0,1900,1993,0.0,2,0.0,6.0,0,...,0,1,0,0,0,1,0,0,0,138500


In [4]:
test.head()

,Id,Total_Bsmt_SF,Gr_Liv_Area,Garage_Area,Year_Built,Year_Remod/Add,Mas_Vnr_Area,Full_Bath,BsmtFin_SF_1,Overall_Qual,...,Kitchen_Qual,Fireplace_Qu,Garage_Finish,Bsmt_Exposure,Heating_QC,MS_SubClass,Foundation,Garage_Type,Neighborhood_High,Neighborhood_Low
0,2658,1020,1928,440,1910,1950,0.0,2,0,6.0,...,0,0,1,0,1,0,0,0,0,1
1,2718,1967,1967,580,1977,1977,0.0,2,0,5.0,...,0,0,3,0,0,0,0,1,0,0
2,2414,654,1496,426,2006,2006,0.0,2,554,7.0,...,1,3,2,2,2,1,1,1,0,0
3,1989,968,968,480,1923,2006,0.0,1,0,5.0,...,0,0,1,0,0,0,0,0,0,1
4,625,1394,1394,514,1963,1963,247.0,1,609,6.0,...,0,3,2,0,1,1,0,1,0,0


In [5]:
# create X dataframe for features and y series for target
X_train = train.drop(columns=['SalePrice'])
y_train = train.SalePrice

X_test = test.drop(columns=['Id'])

In [6]:
# instantiate models
lr = LinearRegression()
lasso = LassoCV(n_alphas=100)
ridge = RidgeCV(alphas=np.linspace(0.1, 10, 100))

In [7]:
# create functions to print cross_val_score for various models with or without scaling

# for default r2
def print_cvs_R2(X, y):
    print('R2 scores')
    for model in [lr, lasso, ridge]:
        print(f'{str(model)[:7]}: {cross_val_score(model, X, y, cv=10).mean()}')
        
# for RMSE
def print_cvs_RMSE(X, y):
    print('RMSE scores')
    for model in [lr, lasso, ridge]:
        print(f'{str(model)[:7]}: {-cross_val_score(model, X, y, cv=10, scoring = "neg_root_mean_squared_error").mean()}')

## Model Selection

### (A) Without Scaling

In [8]:
print_cvs_R2(X_train, y_train)
print()
print_cvs_RMSE(X_train, y_train)

R2 scores
LinearR: 0.8795229053598742
LassoCV: 0.8156379562451843
RidgeCV: 0.8795542150198011

RMSE scores
LinearR: 27423.702267818106
LassoCV: 33896.19733439856
RidgeCV: 27421.440098937495


### (B) Standard Scaler

In [9]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.fit_transform(X_test)

In [10]:
print_cvs_R2(X_train_ss, y_train)
print()
print_cvs_RMSE(X_train_ss, y_train)

R2 scores
LinearR: 0.8795229053598741
LassoCV: 0.8796686766268358
RidgeCV: 0.8795649106949066

RMSE scores
LinearR: 27423.702267818106
LassoCV: 27407.067805104434
RidgeCV: 27419.350038541288


### (C) Power Transform

In [11]:
pt = PowerTransformer()

X_train_pt = pt.fit_transform(X_train)
X_test_pt = pt.fit_transform(X_test)

# fit target values to Power Transform here so we can use pt.inverse_transform directly subsequently
pt.fit(y_train.to_frame())

y_train_pt = pt.transform(y_train.to_frame())

In [12]:
print_cvs_R2(X_train_pt, y_train_pt)
print()
print_cvs_RMSE(X_train_pt, y_train_pt)

R2 scores
LinearR: 0.8806678157166387
LassoCV: 0.8811487671811795
RidgeCV: 0.8807186991862677

RMSE scores
LinearR: 0.342994192332366
LassoCV: 0.342213237353865
RidgeCV: 0.3428999927087064


### Interpreting RMSE scores
The RMSE scores with Power Transform are way smaller than without transformation as we scaled the target values. We will calculate the actual values in the cell below.

In [13]:
def get_RMSE(model):
    model.fit(X_train_pt, y_train_pt)
    model_preds_pt = model.predict(X_train_pt)
    model_preds = pt.inverse_transform(model_preds_pt.reshape(-1, 1))
    return mean_squared_error(y_train, model_preds) ** 0.5

for model in [lr, lasso, ridge]:
    print(f'{str(model)[:7]}: {get_RMSE(model)}')

LinearR: 25148.105660108253
LassoCV: 25361.468682187715
RidgeCV: 25182.721415123375


### Model Selected
The models with Power Transform have the best scores for both R2 and RMSE. Amongst the 3 models, Linear Regression performs slightly better, so we expect it to give the best score for Kaggle.

However, it is difficult to interpret the coefficients of the model after Power Transform, so we will also work on Linear Regression without Power Transform (both with and without Standard Scaling) and interpret the coefficients from these models.

## Generating Predictions using Linear Regression
We will generate predictions on Test Data for Kaggle submissions.

In [14]:
# create function to get predictions

def get_pred(model, X_train, y_train, X_test):
    # fit model
    model.fit(X_train, y_train)
    
    # get predictions
    preds = lr.predict(X_test)
    
    return preds

### (A) Without Scaling

In [15]:
# get_pred
preds_test = get_pred(lr, X_train, y_train, X_test)

# create DataFrame in format required
submission = pd.DataFrame(zip(test.Id, preds_test), columns=['Id', 'SalePrice']).sort_values('Id')

# save to csv
submission.to_csv('datasets/preds.csv', index=False)

### (B) Standard Scaler

In [16]:
# get_pred
preds_test_ss = get_pred(lr, X_train_ss, y_train, X_test_ss)

# create DataFrame in format required
submission_ss = pd.DataFrame(zip(test.Id, preds_test_ss), columns=['Id', 'SalePrice']).sort_values('Id')

# save to csv
submission_ss.to_csv('datasets/preds_ss.csv', index=False)

### (C) Power Transform

In [17]:
# get predictions
preds_test_pt = get_pred(lr, X_train_pt, y_train_pt, X_test_pt)

# inverse transform to get SalePrice
preds_test = pt.inverse_transform(preds_test_pt.reshape(-1, 1))

# create DataFrame in format required by Kaggle
submission_pt = pd.DataFrame(zip(test.Id, preds_test[:,0]), columns=['Id', 'SalePrice']).sort_values('Id')

# save to csv
submission_pt.to_csv('datasets/preds_pt.csv', index=False)

### Kaggle Scores

|Scaling Method|None|Standard Scaler|Power Transform|
|---|---|---|---|
|Private|24813.43949|25191.36174|23599.10771|
|Public|31111.93853|31358.58896|29682.66232|

As expected, Power Transform gave the best scores.

## Evaluating and Interpreting the Models

In [18]:
def get_coef_intercept(model, X, y):
    lr.fit(X, y)
    display(pd.Series(lr.coef_, X_train.columns).sort_values())
    print(f'Model Intercept: {lr.intercept_}')

### (A) Without Scaling

In [19]:
get_coef_intercept(lr, X_train, y_train)

Full_Bath            -4077.486430
Neighborhood_Low     -3283.097101
Garage_Type          -2091.483720
Exter_Qual            -836.451071
Year_Built              -8.738143
Total_Bsmt_SF           18.288548
BsmtFin_SF_1            21.816107
Mas_Vnr_Area            27.459564
Garage_Area             32.149993
Gr_Liv_Area             55.900924
Year_Remod/Add         158.048126
Garage_Finish         1032.229992
Foundation            2163.907765
Fireplace_Qu          2918.696949
Heating_QC            2996.609888
Bsmt_Qual             4994.574648
Bsmt_Exposure         6230.644823
MS_SubClass           9243.532987
Overall_Qual         10302.772816
Kitchen_Qual         14447.822687
Neighborhood_High    31307.240169
dtype: float64

Model Intercept: -337039.15573680284


### (B) Standard Scaler

In [20]:
get_coef_intercept(lr, X_train_ss, y_train)

Full_Bath            -2236.834018
Neighborhood_Low     -1400.392065
Garage_Type           -993.715112
Exter_Qual            -405.890513
Year_Built            -263.598661
Garage_Finish          927.012937
Foundation            1076.819005
Heating_QC            2691.136131
Year_Remod/Add        3322.865943
Bsmt_Qual             3661.984369
Fireplace_Qu          3806.505768
MS_SubClass           4459.575051
Mas_Vnr_Area          4723.266238
Bsmt_Exposure         6518.417739
Garage_Area           6897.357229
Total_Bsmt_SF         7785.552145
Neighborhood_High     8407.505133
Kitchen_Qual          9060.836208
BsmtFin_SF_1          9597.890891
Overall_Qual         14466.910018
Gr_Liv_Area          27000.639062
dtype: float64

Model Intercept: 181464.00929095355


### (C) Power Transform

In [21]:
get_coef_intercept(lr, X_train_pt, y_train_pt[:,0])

Neighborhood_Low    -0.082704
Full_Bath           -0.016519
Exter_Qual          -0.013597
Mas_Vnr_Area        -0.008022
Foundation          -0.001379
Garage_Type          0.005204
Year_Built           0.007598
Garage_Finish        0.018622
MS_SubClass          0.044694
Bsmt_Qual            0.046420
Kitchen_Qual         0.049386
Bsmt_Exposure        0.051767
Neighborhood_High    0.052119
Heating_QC           0.054988
Fireplace_Qu         0.061085
Garage_Area          0.095246
Year_Remod/Add       0.100621
BsmtFin_SF_1         0.116238
Total_Bsmt_SF        0.117682
Overall_Qual         0.215524
Gr_Liv_Area          0.341803
dtype: float64

Model Intercept: -2.027520516152725e-15


### Interpretation of Coefficients
The top 3 features with the largest effects on SalePrice for the various models fitted are:

|Scaling Method|None|Standard Scaler|Power Transform|
|---|---|---|---|
|Top|Neighborhood_High|Gr_Liv_Area|Gr_Liv_Area|
|Second|Kitchen_Qual|Overall_Qual|Overall_Qual|
|Third|Overall_Qual|BsmtFin_SF_1|Total_Bsmt_SF|

No scaling gives very different features from the other 2. This is because the magnitudes of the raw values vary vastly, so one unit increase in Gr_Liv_Area is a lot less noticeable than 1 unit increase in Neighborhood_High.

We should ignore the results of models with no scaling based on this dataset.

The top 2 features are Gr_Liv_Area and Overall_Qual. We will interpret the coefficients using the Standard Scaler version.

In [22]:
# summary statistics for Gr_Liv_Area
X_train[['Gr_Liv_Area','Overall_Qual']].describe()

,Gr_Liv_Area,Overall_Qual
count,2045.000000,2045.000000
mean,1494.266993,6.110024
std,483.126953,1.404520
min,334.000000,3.705000
25%,1128.000000,5.000000
50%,1442.000000,6.000000
75%,1728.000000,7.000000
max,3672.000000,9.214286


In [23]:
# we calculate the equivalent to 1 standardized unit using x = mu + sigma * z
u_1 = 1494.266993 + 483.126953 * 1
u_2 = 6.110024 + 1.404520 * 1
print(f'1 standardized unit of Gr_Liv_Area: {u_1}')
print(f'1 standardized unit of Overall_Qual: {u_2}')

1 standardized unit of Gr_Liv_Area: 1977.393946
1 standardized unit of Overall_Qual: 7.514544


In [24]:
# we then divide the incremental SalePrice ($27000 and $14500) by the above values
# to find the amount SalePrice increases for each square feet increase in Gr_Liv_Area 
# and each 1 point increase in Overall_Qual
m_1 = 27000.639062 / 1977.393946
m_2 = 14466.910018 / 7.514544
print(f'For every 1 square foot increase in Gr_Liv_Area, SalePrice is expected to increase by around ${round(m_1,2)}.')
print(f'For every 1 point increase in Overall_Qual, SalePrice is expected to increase by around ${round(m_2,2)}')

For every 1 square foot increase in Gr_Liv_Area, SalePrice is expected to increase by around $13.65.
For every 1 point increase in Overall_Qual, SalePrice is expected to increase by around $1925.19


## Conclusion
For higher sale price, the most important things to do are to increase to living area above ground and to get a higher score in overall quality by using better material and having better finishing.

Amongst the top ten features in both the models using Standard Scaling and Power Transform, in addition to the 2 already discussed, there are 5 more, of which 3 involve the Basement (BsmtFin_SF_1, Total_Bsmt_SF, Bsmt_Exposure) and the remaining 2 are Neighborhood_High and Garage_Area.

Hence, we would strongly recommend that efforts be put into improving the Basement if we wish to command a higher sale price.

For a house-hunter looking for a better deal, he might wish to avoid the 3 neighborhoods of Brookside, Northridge and Northridge Heights which command higher sale prices than the other neighborhoods in Ames. 